# Optimization 2

In [1]:
from smf.ranker import retrieve
from smf.ranker import rank
from smf.ranker import add_sector_info

import numpy as np
import pandas as pd
import eikon as ek
ek.set_app_key('403255e90c7647afafbfb5c0000d60ac4c8cc536')
from smf.reuters_api import get_tickers

SyntaxError: non-default argument follows default argument (ranker.py, line 67)

# Ranking

In [2]:
closing, pe, pb, pfcf, eebitda, volatilities, sector_info = retrieve(exclude_sector="Financials")

-I- Successful database connection
609 tickers from Financials sector removed


In [26]:
ranked = rank(closing, pe, pb, pfcf, eebitda, volatilities)
final = add_sector_info(ranked, sector_info)

In [7]:
snp500 = get_tickers(date="2019-08-02")

In [28]:
final["Index"] = final.index.isin(snp500)
final["Index"] = final["Index"].apply(lambda x: "S&P" if x else "Russell 2000")

In [29]:
final.head(50)

,combined_score,momentum_score,volatility_score,value_score,volatility,three_month_mom,twelve_month_mom,price_earnings,price_to_book,price_fcf,ev_ebitda,economic_sector,business_sector,industry_group,Index
MAS.N,14.0,4.4,4.8,4.8,14.571172,1.099162,1.535421,65.3671,-96.4549,37.7355,7.98833,Consumer Cyclicals,Cyclical Consumer Products,Homebuilding & Construction Supplies,S&P
MSI.N,13.8,4.4,4.6,4.8,18.546964,1.100521,1.546394,55.8121,-27.9036,30.281,12.9892,Technology,Technology Equipment,Communications & Networking,S&P
CMC.N,13.6,4.2,4.4,5.0,19.890287,1.133684,1.252326,20.2491,1.52296,16.5811,8.52508,Basic Materials,Mineral Resources,Metals & Mining,Russell 2000
ACA.N,13.6,4.8,4.0,4.8,23.420930,1.258777,1.514821,50.998,1.24799,31.0008,7.72538,Industrials,Industrial & Commercial Services,Construction & Engineering,Russell 2000
MHO.N,13.6,4.6,4.0,5.0,24.128695,1.024923,1.846246,28.4559,1.36994,8.5809,7.6245,Consumer Cyclicals,Cyclical Consumer Products,Homebuilding & Construction Supplies,Russell 2000
EVRG.N,13.6,4.2,5.0,4.4,13.081301,1.117601,1.256957,42.6434,1.85615,36.8038,13.087,Utilities,Utilities,Electric Utilities & IPPs,S&P
BMCH.OQ,13.6,4.6,4.4,4.6,19.593949,1.115178,1.836879,52.36,2.16881,37.1829,5.27012,Consumer Cyclicals,Cyclical Consumer Products,Homebuilding & Construction Supplies,Russell 2000
NEM.N,13.4,4.4,4.6,4.4,18.063613,1.132737,1.368952,14.0028,1.69438,122.717,8.28952,Basic Materials,Mineral Resources,Metals & Mining,S&P
BHE.N,13.4,4.6,4.2,4.6,22.296117,1.206649,1.520953,153.302,1.33885,12.8027,4.88707,Technology,Technology Equipment,Semiconductors & Semiconductor Equipment,Russell 2000
NSIT.OQ,13.4,4.6,4.2,4.6,21.587141,1.159241,1.499671,73.6212,2.18769,35.6059,5.46213,Technology,Software & IT Services,Software & IT Services,Russell 2000


# Optimization

In [3]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from smf.db_engine import DbEngine

In [4]:
final_companies1 = [
    "ACA.N",
    "NEM.N",
    "NSIT.OQ",
    "DIOD.OQ",
    "ETN.N",
    "FCN.N",
    "ALCO.OQ",
    "EE.N",
    "PHM.N",
    "PCAR.OQ",
    "ADM.N",
    "CMC.N",
    "BMY.N",
    "DHI.N",
    "HMST.OQ",
    "MS.N",
    "JPM.N",
    "FFWM.OQ",
    "HIG.N",
    "RDN.N",
    "FITB.OQ",
]
len(final_companies1)

21

In [6]:
db = DbEngine()
close_prices = db.fetch_data_series("close_price")

-I- Successful database connection


In [7]:
final_companies = [x.replace(".", "_") for x in final_companies1]

In [10]:
close_prices_3m = close_prices.loc[:"2020-01-24":,final_companies].last("90D").fillna(method="ffill")
close_prices_3m.shape

(61, 21)

In [12]:
min_weight, max_weight = 0.02, 0.1
print("--------------------------------------")
print(f"Weight range: [{min_weight}, {max_weight}]")
print(close_prices_12m.shape)
print()
mu = expected_returns.mean_historical_return(close_prices_12m)
S = risk_models.CovarianceShrinkage(close_prices_12m).ledoit_wolf()

ef = EfficientFrontier(expected_returns=mu, cov_matrix=S, weight_bounds=(min_weight, max_weight))
raw_weights = ef.max_sharpe(risk_free_rate=0.00)
cleaned_weights = ef.clean_weights()

ef.portfolio_performance(verbose=True)
print(pd.Series(cleaned_weights).sort_values(ascending=False))

--------------------------------------
Weight range: [0.02, 0.1]
(61, 21)

Expected annual return: 51.6%
Annual volatility: 5.6%
Sharpe Ratio: 8.80
NEM_N      0.10000
DHI_N      0.10000
ACA_N      0.09578
ALCO_OQ    0.09324
BMY_N      0.08373
NSIT_OQ    0.06630
ETN_N      0.05993
DIOD_OQ    0.05975
EE_N       0.05088
PHM_N      0.03946
CMC_N      0.03326
HIG_N      0.02915
JPM_N      0.02853
FCN_N      0.02000
FITB_OQ    0.02000
PCAR_OQ    0.02000
RDN_N      0.02000
HMST_OQ    0.02000
MS_N       0.02000
FFWM_OQ    0.02000
ADM_N      0.02000
dtype: float64


In [13]:
curr_prices = ek.get_data(final_companies1, "CF_LAST")[0]
curr_prices['Instrument'] = curr_prices['Instrument'].apply(lambda x: x.replace(".", "_"))
curr_prices = curr_prices.set_index("Instrument")['CF_LAST']
da = DiscreteAllocation(cleaned_weights, curr_prices, total_portfolio_value=100000)
allocation, leftover = da.lp_portfolio()
print("Left Over: {}".format(leftover))

0 out of 21 tickers were removed
Left Over: 3.53


In [14]:
df = pd.DataFrame()
df['Number of shares'] = pd.Series(allocation)
df['Weights'] = pd.Series(cleaned_weights).sort_values(ascending=False)
df['Current Price'] = pd.Series(curr_prices)
df.sort_values("Weights", ascending=False)

,Number of shares,Weights,Current Price
NEM_N,205,0.10000,48.750
DHI_N,178,0.10000,56.320
ACA_N,228,0.09578,42.030
ALCO_OQ,284,0.09324,32.820
BMY_N,137,0.08373,61.160
NSIT_OQ,120,0.06630,55.080
ETN_N,63,0.05993,94.500
DIOD_OQ,135,0.05975,44.200
EE_N,75,0.05088,68.110
PHM_N,95,0.03946,41.720


In [15]:
(df['Number of shares'] * df['Current Price']).sum()

99996.47000000002

In [17]:
df.to_csv("optimisation2.csv")